### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [12]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [13]:
reviews.head(20)

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697
5,2,7131622,8,1579559244
6,2,7975244,6,1583706305
7,2,7984734,7,1579949842
8,2,8367814,8,1586436354
9,2,8579674,10,1579261830


In [25]:
genres = set(x for l in list(movies['genre'].dropna().str.split('|')) for x in l)

In [15]:
print(f"The number of movies in the dataset: {movies.shape[0]}")
print(f"The number of ratings in the dataset: {reviews.shape[0]}")
print(f"The number of different genres: {len(set(x for l in list(movies['genre'].dropna().str.split('|')) for x in l))}")
print(f"The number of unique users in the dataset: {len(pd.unique(reviews['user_id']))}")
print(f"The number missing ratings in the reviews dataset: {35479 * 67353 - 863866}")
print(f"The average rating given across all ratings: {reviews['rating'].mean()}")
print(f"The minimum rating given across all ratings: {reviews['rating'].min()}")
print(f"The maximum rating given across all ratings: {reviews['rating'].max()}")

The number of movies in the dataset: 35479
The number of ratings in the dataset: 863866
The number of different genres: 28
The number of unique users in the dataset: 67353
The number missing ratings in the reviews dataset: 2388753221
The average rating given across all ratings: 7.315877693994207
The minimum rating given across all ratings: 0
The maximum rating given across all ratings: 10


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [16]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [17]:
movies["year"] = movies['movie'].apply(lambda x: x.split('(')[1].replace(')', '')).astype('int64')

In [18]:
movies["year_dummy"] = movies.apply(lambda x: "2000's" if x["year"] >= 2000 
                                    else ("1900's" if x["year"] >= 1900 
                                          else ("1800's" if x["year"] >= 1800 else 0)), axis = 1)

In [19]:
movies.head()

,movie_id,movie,genre,year,year_dummy
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1800's
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1800's
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1800's
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1800's
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1800's


In [20]:
s = pd.get_dummies(movies, columns = ['year_dummy'])

In [22]:
s.head()

,movie_id,movie,genre,year,year_dummy_1800's,year_dummy_1900's,year_dummy_2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [27]:
def split_genres(val):
    try:
        if val.find(gene) >-1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

for gene in genres:        
    movies[gene] = movies['genre'].apply(split_genres)

In [29]:
movies.head()

,movie_id,movie,genre,year,year_dummy,Adventure,Thriller,Family,Reality-TV,Comedy,...,Mystery,History,Musical,Short,Horror,Game-Show,War,Animation,Fantasy,Romance
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1800's,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1800's,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1800's,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1800's,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1800's,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [30]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date
0           0        1    114508       8  1381006850  2013-10-05 21:00:50
1           1        2    208092       5  1586466072  2020-04-09 21:01:12
2           2        2    358273       9  1579057827  2020-01-15 03:10:27
3           3        2  10039344       5  1578603053  2020-01-09 20:50:53
4           4        2   6751668       9  1578955697  2020-01-13 22:48:17
   Unnamed: 0  movie_id                                              movie  \
0           0         8      Edison Kinetoscopic Record of a Sneeze (1894)   
1           1        10                La sortie des usines Lumière (1895)   
2           2        12                      The Arrival of a Train (1896)   
3           3        25  The Oxford and Cambridge University Boat Race ...   
4           4        91                         Le manoir du diable (1896)   

               genre  date  1800's  1900's  2000's  Horror  Biography  \
0  Documentary